In [9]:
import timeit
import torch
import numpy as np

# import qpth.solvers.dynamic.solve as dynamic_solver
from qpth.util import get_sizes, extract_nBatch, expandParam, bdiag
from qpth.solvers import cvxpy, dynamic
from qpth.qp import QPFunction, QPSolvers

In [10]:
def get_kkt_problem(nBatch=2, nx=5, nineq=4, neq=3):
    def cast(m):
        # return m.cuda().double()
        return m.double()

    Q = cast(torch.randn(nx, nx))
    Q = Q.mm(Q.t())
    p = cast(torch.randn(nx))
    G = cast(torch.randn(nBatch, nineq, nx))
    h = cast(torch.randn(nBatch, nineq))
    A = cast(torch.randn(neq, nx))
    b = cast(torch.randn(neq))

    nBatch = extract_nBatch(Q, p, G, h, A, b)
    Q, _ = expandParam(Q, nBatch, 3)
    p, _ = expandParam(p, nBatch, 2)
    G, _ = expandParam(G, nBatch, 3)
    h, _ = expandParam(h, nBatch, 2)
    A, _ = expandParam(A, nBatch, 3)
    b, _ = expandParam(b, nBatch, 2)

    return Q, p, G, h, A, b

In [27]:
# Full Constraints

N_ITERS = 100

pdipm = QPFunction(verbose=-1, maxIter=20, solver=QPSolvers.PDIPM_BATCHED)
dynamic = QPFunction(verbose=-1, maxIter=100, solver=QPSolvers.DYNAMIC)
cvxpy = QPFunction(verbose=-1, maxIter=20, solver=QPSolvers.CVXPY)

num_same = 0
num_infeas = 0

time_p = 0
time_d = 0

showed = False
failed = False

for _ in range(N_ITERS):
    Q, p, G, h, A, b = get_kkt_problem(1, 4, 3, 2)

    # Check for infeasibility with CVXPY
    try:
        zc = cvxpy(Q, p, G, h, A, b)
    except:
        num_infeas += 1
        continue

    start = timeit.default_timer()
    zp = pdipm(Q, p, G, h, A, b)

    mid = timeit.default_timer()
    zd = dynamic(Q, p, G, h, A, b)

    stop = timeit.default_timer()
    time_p += mid - start
    time_d += stop - mid

    if np.allclose(zp.cpu(), zd.cpu(), rtol=1e-2):
        num_same += 1
        if not showed:
            showed = True
            print("Success Case:")
            print(Q, p, G, h, A, b, zp)
    elif not failed:
        failed = True
        print("Fail Case:")
        print(Q, p, G, h, A, b, zp, zd)
        

print(num_same, num_infeas)
print(time_p, time_d)

Fail Case:
tensor([[[ 1.6947,  1.5521,  1.2133, -0.7394],
         [ 1.5521,  2.3145,  2.3238, -2.4168],
         [ 1.2133,  2.3238,  3.8217, -1.8617],
         [-0.7394, -2.4168, -1.8617,  4.5391]]], dtype=torch.float64) tensor([[ 2.1825, -0.4478,  0.7832,  0.1248]], dtype=torch.float64) tensor([[[-1.1516,  1.5950,  0.3905,  0.5312],
         [-1.6532, -0.7834, -0.2246, -0.5737],
         [ 0.5059, -0.5417,  0.0481, -0.8444]]], dtype=torch.float64) tensor([[-0.1556, -0.0132,  1.5622]], dtype=torch.float64) tensor([[[ 0.4470, -0.2505,  0.7365,  1.5065],
         [ 0.4179,  0.0945, -1.6606, -0.0292]]], dtype=torch.float64) tensor([[ 2.0092, -1.2711]], dtype=torch.float64) tensor([[ 0.1360, -0.4664,  0.7583,  0.8450]], dtype=torch.float64) tensor([[-3.7631e+252,  6.4346e+252, -1.6850e+252,  2.1231e+252]],
       dtype=torch.float64)
Success Case:
tensor([[[ 3.6341, -0.5191, -1.4358,  1.8919],
         [-0.5191,  1.5365,  0.1738,  0.2076],
         [-1.4358,  0.1738,  0.8882, -0.8012],
  

In [26]:
# Ineq Constraints

N_ITERS = 100

pdipm = QPFunction(verbose=-1, maxIter=20, solver=QPSolvers.PDIPM_BATCHED)
dynamic = QPFunction(verbose=-1, maxIter=100, solver=QPSolvers.DYNAMIC_INEQ)
cvxpy = QPFunction(verbose=-1, maxIter=20, solver=QPSolvers.CVXPY)

num_same = 0
num_infeas = 0

time_p = 0
time_d = 0

showed = False
failed = False

for _ in range(N_ITERS):
    Q, p, G, h, A, b = get_kkt_problem(1, 3, 2, 0)

    # Check for infeasibility with CVXPY
    try:
        zc = cvxpy(Q, p, G, h, A, b)
    except:
        num_infeas += 1
        continue

    start = timeit.default_timer()
    zp = pdipm(Q, p, G, h, A, b)

    mid = timeit.default_timer()
    zd = dynamic(Q, p, G, h, A, b)

    stop = timeit.default_timer()
    time_p += mid - start
    time_d += stop - mid

    if np.allclose(zp.cpu(), zd.cpu(), rtol=1e-2):
        num_same += 1
        if not showed:
            showed = True
            print("Success Case:")
            print(Q, p, G, h, zp)
    elif not failed:
        failed = True
        print("Fail Case:")
        print(Q, p, G, h, zp, zd)

print(num_same, num_infeas)
print(time_p, time_d)

Success Case:
tensor([[[ 1.2111, -0.4908,  0.1365],
         [-0.4908,  2.5101,  1.3620],
         [ 0.1365,  1.3620,  1.4972]]], dtype=torch.float64) tensor([[ 0.6489,  1.2824, -1.3994]], dtype=torch.float64) tensor([[[ 1.4043,  0.6624,  0.3864],
         [ 1.6231, -0.1711, -1.6594]]], dtype=torch.float64) tensor([[-1.0521,  0.3671]], dtype=torch.float64) tensor([[-2.2337, -3.0911,  3.9503]], dtype=torch.float64)
Fail Case:
tensor([[[ 3.9248,  0.4327, -1.1629],
         [ 0.4327,  1.2784,  1.9575],
         [-1.1629,  1.9575,  4.0463]]], dtype=torch.float64) tensor([[1.3726, 0.2471, 2.1340]], dtype=torch.float64) tensor([[[ 0.3656,  1.0899,  0.2022],
         [-1.4219,  0.0922, -1.4658]]], dtype=torch.float64) tensor([[-0.0597,  0.6467]], dtype=torch.float64) tensor([[-0.1499,  0.0498, -0.2927]], dtype=torch.float64) tensor([[ -7.2232,  24.0613, -14.2438]], dtype=torch.float64)
61 1
1.7423277000004873 0.40921929999854
